In [5]:
# Import data

In [3]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class importDataset(Dataset):
    def __init__(self, tiles, subtiles, neighbor_tiles, labels, meta_info=None, transform=None):
        """
        tiles: List of center tiles (np.array, H×W×3)
        subtiles: List of list (9 subtiles per spot), each subtile is np.array(H', W', 3)
        neighbor_tiles: List of list (8 neighbor tiles per spot), each tile is np.array(H, W, 3)
        labels: List of np.array (35,)
        meta_info: Optional list of (slide_id, x, y) for each sample
        transform: torchvision transform to apply to all tiles
        """
        self.tiles = tiles
        self.subtiles = subtiles
        self.neighbor_tiles = neighbor_tiles
        self.labels = labels
        self.meta_info = meta_info if meta_info is not None else [None] * len(tiles)

        self.transform = transform or transforms.Compose([
            transforms.ToTensor(),  # (H, W, C) → (C, H, W)
            transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
        ])

    def __len__(self):
        return len(self.tiles)

    def __getitem__(self, idx):
        center_tile = self.transform(self.tiles[idx])  # → (3, H, W)

        subtiles = [self.transform(t) for t in self.subtiles[idx]]  # → list of (3, h, w)
        subtiles = torch.stack(subtiles)  # → (9, 3, h, w)

        neighbor_tiles = [self.transform(t) for t in self.neighbor_tiles[idx]]  # → list of (3, H, W)
        neighbor_tiles = torch.stack(neighbor_tiles)  # → (8, 3, H, W)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)  # → (35,)
        meta = self.meta_info[idx]

        return {
            'center_tile': center_tile,              # (3, H, W)
            'subtiles': subtiles,                    # (9, 3, h, w)
            'neighbor_tiles': neighbor_tiles,        # (8, 3, H, W)
            'label': label,                          # (35,)
            'meta': meta
        }


In [4]:
# 載入資料
data = torch.load("train_dataset.pt")
train_dataset = importDataset(
    tiles=data['tiles'],
    labels=data['labels'],
    subtiles=data['subtiles'],
    neighbor_tiles=data['neighbor_tiles'],
    meta_info=data['meta_info']
)
# 載入資料
data = torch.load("test_dataset.pt")
test_dataset = importDataset(data['tiles'], data['subtiles'], data['neighbor_tiles'], data['labels'], data['meta_info'])

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_10837/2444037513.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("train_dataset.pt")
/var/fo

In [6]:
def check_dataset_item(dataset, idx=0):
    item = dataset[idx]

    print("🔍 Checking dataset sample:", idx)

    tile = item['center_tile']
    print(f"📏 Center tile shape: {tile.shape} | dtype: {tile.dtype} | min: {tile.min():.3f}, max: {tile.max():.3f}")
    assert tile.ndim == 3 and tile.shape[0] == 3, "❌ Center tile shape 不正確，應為 (3, H, W)"

    subtiles = item['subtiles']
    print(f"📏 Subtiles shape: {subtiles.shape} | dtype: {subtiles.dtype}")
    assert subtiles.shape[1] == 3, "❌ Subtile shape 不正確，應為 (9, 3, h, w)"

    neighbors = item['neighbor_tiles']
    print(f"📏 Neighbor tiles shape: {neighbors.shape} | dtype: {neighbors.dtype}")
    assert neighbors.shape[1] == 3, "❌ Neighbor tile shape 不正確，應為 (8, 3, H, W)"

    label = item['label']
    print(f"🧬 Label shape: {label.shape} | dtype: {label.dtype}")
    assert label.shape[0] == 35 and label.dtype == torch.float32, "❌ Label 應為 float32 且長度為 35"

    print("✅ All checks passed!")
    
check_dataset_item(train_dataset, idx=0)
check_dataset_item(test_dataset, idx=10)



🔍 Checking dataset sample: 0
📏 Center tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: -0.192, max: 0.953
📏 Subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32
📏 Neighbor tiles shape: torch.Size([8, 3, 78, 78]) | dtype: torch.float32
🧬 Label shape: torch.Size([35]) | dtype: torch.float32
✅ All checks passed!
🔍 Checking dataset sample: 10
📏 Center tile shape: torch.Size([3, 78, 78]) | dtype: torch.float32 | min: -0.388, max: 1.000
📏 Subtiles shape: torch.Size([9, 3, 26, 26]) | dtype: torch.float32
📏 Neighbor tiles shape: torch.Size([8, 3, 78, 78]) | dtype: torch.float32
🧬 Label shape: torch.Size([35]) | dtype: torch.float32
✅ All checks passed!


# model 